In [7]:
from pprint import pprint
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

from smolagents import CodeAgent

# Telemetry

Run Phoenix server with
```
phoenix serve
```

In [2]:
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

from phoenix.otel import register

endpoint = "http://0.0.0.0:6006/v1/traces"
# trace_provider = TracerProvider() # this is from HF page.
trace_provider = register(
    project_name="smolagents-project",
    endpoint=endpoint,
    set_global_tracer_provider=False
)
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

/Users/pisek/miniconda3/envs/smol/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: smolagents-project
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://0.0.0.0:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



# Model

In [3]:
from smolagents import OpenAIServerModel

model_openai = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

# Web Tools

## Web Interaction Tools

In [4]:
from smolagents import tool

@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Searches for text on the current page via Ctrl + F and jumps to the nth occurrence.
    Args:
        text: The text to search for
        nth_result: Which occurrence to jump to (default: 1)
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"Match n°{nth_result} not found (only {len(elements)} matches found)")
    result = f"Found {len(elements)} matches for '{text}'."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"Focused on element {nth_result} of {len(elements)}"
    return result

@tool
def go_back() -> None:
    """Goes back to previous page."""
    driver.back()

@tool
def close_popups() -> str:
    """
    Closes any visible modal or pop-up on the page. Use this to dismiss pop-up windows!
    This does not work on cookie consent banners.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()

## Selenium

In [5]:
import helium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from smolagents.agents import ActionStep

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--force-device-scale-factor=1")
chrome_options.add_argument("--window-size=1000,1350")
chrome_options.add_argument("--disable-pdf-viewer")
chrome_options.add_argument("--window-position=0,0")

# Initialize the browser
driver = helium.start_chrome(headless=False, options=chrome_options)

## Screenshot Callback

In [8]:
from time import sleep
from PIL import Image
from io import BytesIO

# Set up screenshot callback
def save_screenshot(memory_step: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # Let JavaScript animations happen before taking the screenshot
    driver = helium.get_driver()
    current_step = memory_step.step_number
    if driver is not None:
        for previous_memory_step in agent.memory.steps:  # Remove previous screenshots for lean processing
            if isinstance(previous_memory_step, ActionStep) and previous_memory_step.step_number <= current_step - 2:
                previous_memory_step.observations_images = None
        png_bytes = driver.get_screenshot_as_png()
        image = Image.open(BytesIO(png_bytes))
        print(f"Captured a browser screenshot: {image.size} pixels")
        memory_step.observations_images = [image.copy()]  # Create a copy to ensure it persists

    # Update observations with current URL
    url_info = f"Current url: {driver.current_url}"
    memory_step.observations = (
        url_info if memory_step.observations is None else memory_step.observations + "\n" + url_info
    )

# Agent

In [9]:
from smolagents import CodeAgent, DuckDuckGoSearchTool


agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), search_item_ctrl_f, go_back, close_popups],
    model=model_openai,
    max_steps=20,
    verbosity_level=2,
    additional_authorized_imports=["helium"],
    step_callbacks=[save_screenshot],
)

agent.python_executor("from helium import *", agent.state)

(None, '', False)

In [10]:
helium_instructions = """
You can use helium to access websites. Don't bother about the helium driver, it's already managed.
We've already ran "from helium import *"
Then you can go to pages!
Code:
```py
go_to('github.com/trending')
```<end_code>

You can directly click clickable elements by inputting the text that appears on them.
Code:
```py
click("Top products")
```<end_code>

If it's a link:
Code:
```py
click(Link("Top products"))
```<end_code>

If you try to interact with an element and it's not found, you'll get a LookupError.
In general stop your action after each button click to see what happens on your screenshot.
Never try to login in a page.

To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.
Code:
```py
scroll_down(num_pixels=1200) # This will scroll one viewport down
```<end_code>

When you have pop-ups with a cross icon to close, don't try to click the close icon by finding its element or targeting an 'X' element (this most often fails).
Just use your built-in tool `close_popups` to close them:
Code:
```py
close_popups()
```<end_code>

You can use .exists() to check for the existence of an element. For example:
Code:
```py
if Text('Accept cookies?').exists():
    click('I accept')
```<end_code>
"""

In [11]:
search_request = """
What is Moodeng? Search for links, gather info from top 3 links, and summarize for me.
When you gather info from links, do one link at a time, and use helium to interact with the link to access website.
"""

# search_request = """
# I want to see the sentiment of the stock markets, focusing on major ones.
# Can you search and browse? Summarize what you find for me.
# """

In [12]:
payload = search_request+" "+helium_instructions

In [13]:
response = agent.run(payload, reset=True)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Moodeng? Search for links, gather info from top 3 links, and summarize for me.                          │
│ When you gather info from links, do one link at a time, and use helium to interact with the link to access      │
│ website.                                                                                                        │
│                                                                                                                 │
│ You can use helium to access websites. Don't bother about the helium driver, it's already managed.              │
│ We've already ran "from helium import *"                                                                        │
│ Then you can go to pages!                                                                                       │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ go_to('github.com/trending')                                                                                    │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ You can directly click clickable elements by inputting the text that appears on them.                           │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click("Top products")                                                                                           │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ If it's a link:                                                                                                 │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click(Link("Top products"))                                                                                     │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ If you try to interact with an element and it's not found, you'll get a LookupError.                            │
│ In general stop your action after each button click to see what happens on your screenshot.                     │
│ Never try to login in a page.                                                                                   │
│                                                                                                                 │
│ To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.     │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ scroll_down(num_pixels=1200) # This will scroll one viewport down                                               │
│ ```<end_code>                                         

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To gather information about Moodeng, I will perform a web search to find links related to it. After that, 
I will access the top three links, extract the relevant information from each, and summarize the findings.         
                                                                                                                   
First, I will start with a web search to find relevant links for "Moodeng".                                        
Code:                                                                                                              
```py                                                                                                              
search_results = web_search(query="Moodeng")                                                                       
print(search_results)                                                                                              
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="Moodeng")                                                                     
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Moo Deng - Wikipedia](https://en.wikipedia.org/wiki/Moo_Deng)
Moo Deng was born on 10 July 2024 to parents Tony and Jonah. She has two full siblings (Nadet and Moo Tun) and four
half-siblings (Ko, Kanya, Phalo, and Moo Wan). [3] The name Moo Deng was chosen through a public poll, with over 
20,000 people voting for it. It translates to ' bouncy pork ' or ' bouncy pig ' in English. [4] [5] [6] [7]

[Moo Deng (moodengsol.com) price today, MOODENG to USD live price 
...](https://coinmarketcap.com/currencies/moo-deng-solana/)
We update our MOODENG to USD price in real-time. Moo Deng (moodengsol.com) is down 8.52% in the last 24 hours. The 
current CoinMarketCap ranking is #526, with a live market cap of $59,844,786 USD. It has a circulating supply of 
989,971,791 MOODENG coins and a max. supply of 989,971,791 MOODENG coins.

[Moo Deng Animal Charity](https://moodeng.org/)
Moo Deng Animal Charity is a volunteer organization that protects and rescues endangered species and street 
animals. It is named after Moo Deng, a dwarf hippo, and operates without fundraising or donations.

[Moo Deng | Solana](https://www.moodengsol.com/)
Moo Deng is a pygmy hippopotamus living in Khao Kheow Open Zoo in Thailand. She is a popular Internet meme and now 
lives on the solana blockchain as a fun and cute token. Learn how to buy, swap, and create your own Moo Deng PFP.

[Moo Deng Price: MOODENG Live Price Chart, Market Cap & News Today 
...](https://www.coingecko.com/en/coins/moo-deng)
The lowest price paid for Moo Deng (MOODENG) is BTC0.0 6 2456, which was recorded on Sep 20, 2024 (5 months). 
Comparatively, the current price is 270.60% higher than the all-time low price. What is the market cap of Moo Deng 
(MOODENG)? Market capitalization of Moo Deng (MOODENG) is BTC577.2222 and is ranked #676 on CoinGecko today. Market
cap ...

[MOODENGUSDT Charts and Quotes — TradingView](https://www.tradingview.com/symbols/MOODENGUSDT/)
Moodeng bullish scenario Moodeng price has broken down from the macro trend and looks to have found a bottom. 
Bullish price action has been formed with what appears to be consolidation over the last 2 weeks. Good area to 
enter a long with invalidation below the previous low. Upside potential makes a good RR. Alternatively.

[MOODENG to USD: Moo Deng Price in US Dollar - CoinGecko](https://www.coingecko.com/en/coins/moo-deng/usd)
The conversion rate of Moo Deng (MOODENG) to USD is $0.05833 for every 1 MOODENG. This means you can exchange 5 
MOODENG for $0.291627 or $50.00 for 857.26 MOODENG, excluding fees. Refer to our conversion tables for popular 
MOODENG trading amounts in their corresponding USD prices and vice versa.

[Moo Deng price: MOODENG to USD, chart & market stats](https://crypto.news/price/moo-deng/)
The current price of Moo Deng (MOODENG) is $0.060308, as of Feb 21, 2025. Over the last 24 hours, the price has 
moved by 6.5%, with a change of 0.03% in the past hour. For longer-term performance, the price has varied by 
-16.43% over the last 7 days and by -59.96% in the past month. Market update

[Moo Deng(MOODENG) Price Today, MOODENG to USD Real-Time, Latest ...](https://stocktwits.com/symbol/MOODENG.X)
Track Moo Deng(MOODENG) Price, real-time chart movements, latest community messages, chart, news and related 
information. Share your ideas and get valuable insights from the community of like minded traders and investors

[Moodeng](https://moodeng.vip/)
MOODENG is an ERC-20 token that celebrates Moodeng, a pygmy hippo born in 2024 and adopted by Vitalik Buterin. 
Learn about its history, roadmap, partnerships, and charity work in the crypto space.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I've accessed the Moo Deng Animal Charity website. Here are the key points extracted:                     
                                                                                                                   
- **Moo Deng Animal Charity** is named after Moo Deng, a dwarf hippopotamus, which is an endangered species.       
- The organization is committed to protecting and rescuing Moo Deng, other endangered species, and street animals  
in need.                                                                                                           
- All activities of the charity are funded solely through personal contributions from its volunteers; the charity  
does not engage in fundraising or accept donations.                                                                
                                                                                                                   
Now I have gathered information from the three links. I'll summarize the findings.                                 
                                                                                                                   
**Summary of Moodeng:**                                                                                            
                                                                                                                   
1. **Moo Deng**:                                                                                                   
   - A female pygmy hippo born on July 10, 2024, in Thailand.                                                      
   - Became an internet meme due to her popularity online.                                                         
   - The name translates to 'bouncy pork' or 'bouncy pig', chosen through a public vote.                           
                                                                                                                   
2. **Moo Deng Cryptocurrency**:                                                                                    
   - Currently priced at approximately **$0.06434**.                                                               
   - Experienced a price change of **6.81%** in the last day.                                                      
                                                                                                                   
3. **Moo Deng Animal Charity**:                                                                                    
   - A volunteer organization focused on protecting endangered species and street animals.                         
   - Operates without fundraising; activities are solely funded by volunteer contributions.                        
                                                                                          

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Moo Deng is a pygmy hippo from Thailand, a cryptocurrency priced at $0.06434, and an animal        
  charity dedicated to protecting endangered species.")                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Moo Deng is a pygmy hippo from Thailand, a cryptocurrency priced at $0.06434, and an animal 
charity dedicated to protecting endangered species.

Captured a browser screenshot: (1000, 916) pixels


[Step 5: Duration 7.17 seconds| Input tokens: 252,328 | Output tokens: 865]

In [14]:
response

'Moo Deng is a pygmy hippo from Thailand, a cryptocurrency priced at $0.06434, and an animal charity dedicated to protecting endangered species.'